In [1]:
#import necessary packages
import pandas as pd 
import splitfolders

from keras.preprocessing.image import ImageDataGenerator

from keras import models
from keras import layers
from keras import optimizers
from keras import metrics

In [ ]:
#use the splitfolders package to split our image data into train, validation, and test sets.
#this is now commented out so that we don't create a new folder each time we run the notebook

#splitfolders.ratio("Data", output="Data_Split",
#    seed=42, ratio=(.64, .16, .2), group_prefix=None, move=True)

In [2]:
#create DataGenerator objects for each set with rescaling for maximum pixel value of 255
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
#create a train_generator from the training directory that has been created from the split folders package
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        'Data_Split/train',
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=4280,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary', 
        color_mode='grayscale')

Found 3749 images belonging to 2 classes.


In [4]:
#create a validation_generator from the training directory that has been created from the split folders package
validation_generator = val_datagen.flow_from_directory('Data_Split/val',
                                                        target_size=(150, 150),
                                                        batch_size=978,
                                                        class_mode='binary',
                                                        color_mode='grayscale')

Found 936 images belonging to 2 classes.


In [5]:
#create a test_generator from the training directory that has been created from the split folders package
test_generator = test_datagen.flow_from_directory('Data_Split/test',
                                                  target_size=(150, 150),
                                                  batch_size=1173,
                                                  class_mode='binary',
                                                  color_mode='grayscale')

Found 1107 images belonging to 2 classes.


In [6]:
train_data, train_labels = next(train_generator)

In [7]:
train_data.shape

(3749, 150, 150, 1)

In [ ]:
train_labels.shape

In [8]:
train_data = train_data.reshape(train_data.shape[0], -1)

In [ ]:
train_data.shape

In [9]:
baseline_model = models.Sequential()

In [10]:
baseline_model.add(layers.Dense(12, activation='relu', input_shape=(22500,)))
baseline_model.add(layers.Dense(1, activation='sigmoid'))

In [11]:
baseline_model.compile(optimizer='SGD', 
                       loss='binary_crossentropy',
                       metrics=['accuracy', metrics.Precision(), metrics.Recall()])

In [12]:
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                270012    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 13        
Total params: 270,025
Trainable params: 270,025
Non-trainable params: 0
_________________________________________________________________


In [13]:
baseline_fit = baseline_model.fit(train_data,
                                  train_labels,
                                  epochs=5,
                                  batch_size=32,
                                  #validation_data=validation_generator,
                                  verbose=1)

Epoch 1/5
118/118 [==============================] - 0s 3ms/step - loss: 0.5676 - accuracy: 0.7359 - precision: 0.7434 - recall: 0.9744
Epoch 2/5
118/118 [==============================] - 0s 3ms/step - loss: 0.3873 - accuracy: 0.8392 - precision: 0.8468 - recall: 0.9518
Epoch 3/5
118/118 [==============================] - 0s 3ms/step - loss: 0.3325 - accuracy: 0.8602 - precision: 0.8846 - recall: 0.9298
Epoch 4/5
118/118 [==============================] - 0s 3ms/step - loss: 0.2741 - accuracy: 0.8880 - precision: 0.9142 - recall: 0.9342
Epoch 5/5
118/118 [==============================] - 0s 3ms/step - loss: 0.2722 - accuracy: 0.8882 - precision: 0.9118 - recall: 0.9375
